In [141]:
% load_ext autoreload
% autoreload 2

UsageError: Line magic function `%` not found.


In [142]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

In [143]:
from shared.paths import DatasetPath

DS = DatasetPath('house-of-representatives-congress-116')

In [144]:
spark = (SparkSession.builder
         .appName(f'{DS}')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .config("spark.executor.memory", "8g")
         .config("spark.driver.memory", "8g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .getOrCreate())

In [145]:
udf_eval_array = F.udf(lambda x: eval(x), T.ArrayType(T.StringType()))

In [146]:
df_legislation = (
    spark.read.csv(DS.raw_str('house_legislation_116.csv'), header=True, inferSchema=True, quote='"', escape='"')
        .withColumn('date_introduced', F.to_timestamp(F.col('date_introduced'), 'yyyy-MM-dd'))
        .withColumn('cosponsors', udf_eval_array(F.col('cosponsors')))
        .withColumn('related_bills', udf_eval_array(F.col('related_bills')))
        .withColumn('subjects', udf_eval_array(F.col('subjects')))
        .withColumn('committees', udf_eval_array(F.col('committees')))
)
df_legislation.head(5)

[Row(bill_id='H.R.1', title="To expand Americans' access to the ballot box, reduce the influence of big money in politics, and strengthen ethics rules for public servants, and for other purposes.", sponsor='S001168', cosponsors=['P000197', 'A000370', 'A000376', 'B001300', 'B001281', 'B001292', 'B000490', 'B000574', 'B001278', 'B001296', 'B001304', 'B001285', 'B001286', 'B001251', 'C001112', 'C001097', 'C001090', 'C001055', 'C001117', 'C001066', 'C001080', 'C001084', 'C001123', 'C001101', 'C001067', 'C001049', 'C000537', 'C001068', 'C001078', 'C000754', 'C001110', 'C001069', 'C001124', 'C001119', 'C001111', 'C001121', 'C000984', 'D000629', 'D000096', 'D000598', 'D000631', 'D000191', 'D000197', 'D000216', 'D000617', 'D000630', 'D000627', 'D000623', 'D000610', 'D000624', 'D000482', 'E000179', 'E000299', 'E000215', 'E000297', 'E000296', 'F000468', 'F000462', 'G000571', 'G000574', 'G000559', 'G000587', 'G000586', 'G000592', 'G000585', 'G000553', 'G000551', 'H001080', 'H001090', 'H000324', '

In [147]:
df_members = (
    spark.read.csv(DS.raw_str('house_members_116.csv'), header=True, inferSchema=True, quote='"', escape='"')
        .withColumn('committee_assignments', udf_eval_array(F.col('committee_assignments')))
)
df_members.head(5)

[Row(name_id='A000374', name='ralph-abraham', state='Louisiana', url='https://www.congress.gov/member/ralph-abraham/A000374', chamber='House', current_party='Republican', committee_assignments=['Agriculture', 'Armed Services']),
 Row(name_id='A000370', name='alma-adams', state='North Carolina', url='https://www.congress.gov/member/alma-adams/A000370', chamber='House', current_party='Democratic', committee_assignments=['Agriculture', 'Education and Labor', 'Financial Services']),
 Row(name_id='A000055', name='robert-aderholt', state='Alabama', url='https://www.congress.gov/member/robert-aderholt/A000055', chamber='House', current_party='Republican', committee_assignments=['Appropriations']),
 Row(name_id='A000371', name='pete-aguilar', state='California', url='https://www.congress.gov/member/pete-aguilar/A000371', chamber='House', current_party='Democratic', committee_assignments=['Appropriations', 'House Administration']),
 Row(name_id='A000372', name='rick-allen', state='Georgia', url

In [148]:
df_rollcall_info = (
    spark.read.csv(DS.raw_str('house_rollcall_info_116.csv'), header=True, inferSchema=True)
        .withColumn('date', F.concat(F.col('date'), F.lit(' '), F.col('year')))
        .withColumn('date', F.to_timestamp(F.col('date'), 'd-MMM yyyy'))
        .drop('year')
)
df_rollcall_info.head(5)

[Row(rollcall_id='2019:001', roll_num=1, date=datetime.datetime(2019, 1, 3, 0, 0), bill_id='QUORUM', question='Call by States', result='P', description=None, congress=116, session=1),
 Row(rollcall_id='2019:002', roll_num=2, date=datetime.datetime(2019, 1, 3, 0, 0), bill_id=None, question='Election of the Speaker', result='P', description='\xa0', congress=116, session=1),
 Row(rollcall_id='2019:003', roll_num=3, date=datetime.datetime(2019, 1, 3, 0, 0), bill_id='H.RES.5', question='On Motion to Table the Motion to Refer', result='P', description='Providing for consideration of H.Res. 6, adopting the Rules of the House of Representatives for the 116th Congress; providing for consideration of H.R. 21, making appropriations for the FY ending September 30, 2019, and for other purposes', congress=116, session=1),
 Row(rollcall_id='2019:004', roll_num=4, date=datetime.datetime(2019, 1, 3, 0, 0), bill_id='H.RES.5', question='On Ordering the Previous Question', result='P', description='Providi

In [149]:
def to_explode(df, by):
    # Filter dtypes and split into column names and type description
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    # Spark SQL supports only homogeneous columns
    assert len(set(dtypes)) == 1, "All columns have to be of the same type"

    # Create and explode an array of (column_name, column_value) structs
    kvs = F.explode(F.array([
        F.struct(F.lit(c).alias("rollcall_id"), F.col(c).alias("vote")) for c in cols
    ])).alias("kvs")

    return df.select(by + [kvs]).select(by + ["kvs.rollcall_id", "kvs.vote"])

In [150]:
udf_vote_val = F.udf(lambda x: (1 if x in ["Aye", "Yea"] else -1 if x in ["No", "Nay"] else 0), T.IntegerType())

In [151]:
df_rollcall_votes = to_explode(
    spark.read.csv(DS.raw_str('house_rollcall_votes_116.csv'), header=True, inferSchema=True),
    ['name_id'],
).cache().withColumn('vote', udf_vote_val(F.col('vote')))
df_rollcall_votes.head(5)

22/04/06 20:57:41 WARN CacheManager: Asked to cache already cached data.


[Row(name_id='A000374', rollcall_id='2019:641', vote=-1),
 Row(name_id='A000374', rollcall_id='2019:639', vote=1),
 Row(name_id='A000374', rollcall_id='2019:640', vote=-1),
 Row(name_id='A000374', rollcall_id='2019:635', vote=1),
 Row(name_id='A000374', rollcall_id='2019:636', vote=1)]

# Construct Graph Entities

In [152]:
df_raw_nodes_members = (
    df_members
        .withColumnRenamed('name_id', 'id')
        .dropDuplicates(['id'])
        .withColumnRenamed('id', 'mid')
        .coalesce(1)
        .withColumn('id', F.monotonically_increasing_id())
)
print(df_raw_nodes_members.count())
df_raw_nodes_members.show(5)

443
+-------+---------------+--------------+--------------------+-------+-------------+---------------------+---+
|    mid|           name|         state|                 url|chamber|current_party|committee_assignments| id|
+-------+---------------+--------------+--------------------+-------+-------------+---------------------+---+
|A000055|robert-aderholt|       Alabama|https://www.congr...|  House|   Republican|     [Appropriations]|  0|
|A000367|   justin-amash|      Michigan|https://www.congr...|  House|  Independent|                   []|  1|
|A000369|    mark-amodei|        Nevada|https://www.congr...|  House|   Republican|     [Appropriations]|  2|
|A000370|     alma-adams|North Carolina|https://www.congr...|  House|   Democratic| [Agriculture, Edu...|  3|
|A000371|   pete-aguilar|    California|https://www.congr...|  House|   Democratic| [Appropriations, ...|  4|
+-------+---------------+--------------+--------------------+-------+-------------+---------------------+---+
only s

In [153]:
parties_list = df_raw_nodes_members.groupby('current_party').count().filter(F.col('count') > 5).select(
    'current_party').rdd.flatMap(lambda x: x).collect()
print(len(parties_list))
print(parties_list)

state_list = df_raw_nodes_members.groupby('state').count().filter(F.col('count') > 6).select('state').rdd.flatMap(
    lambda x: x).collect()
print(len(state_list))
print(state_list)

2
['Republican', 'Democratic']
24
['Alabama', 'Michigan', 'North Carolina', 'California', 'Georgia', 'Texas', 'Florida', 'Indiana', 'Ohio', 'Illinois', 'Virginia', 'Pennsylvania', 'Colorado', 'Arizona', 'Maryland', 'New York', 'Tennessee', 'South Carolina', 'Missouri', 'Massachusetts', 'Minnesota', 'Wisconsin', 'Washington', 'New Jersey']


In [154]:
df_nodes_members = (
    df_raw_nodes_members.select(
        ['*'] + [
            (F.col('current_party') == party).alias(f'feat_{party}')
            for party in parties_list
        ] + [
            (F.col('state') == state).alias(f'feat_' + state.replace(' ', ''))
            for state in state_list
        ] + [
            (~F.col('state').isin(state_list)).alias(f'feat_' + 'StateOther'),
            (~F.col('current_party').isin(parties_list)).alias(f'feat_' + 'PartyOther'),
        ]
    )
)
df_nodes_members.head(5)

[Row(mid='A000055', name='robert-aderholt', state='Alabama', url='https://www.congress.gov/member/robert-aderholt/A000055', chamber='House', current_party='Republican', committee_assignments=['Appropriations'], id=0, feat_Republican=True, feat_Democratic=False, feat_Alabama=True, feat_Michigan=False, feat_NorthCarolina=False, feat_California=False, feat_Georgia=False, feat_Texas=False, feat_Florida=False, feat_Indiana=False, feat_Ohio=False, feat_Illinois=False, feat_Virginia=False, feat_Pennsylvania=False, feat_Colorado=False, feat_Arizona=False, feat_Maryland=False, feat_NewYork=False, feat_Tennessee=False, feat_SouthCarolina=False, feat_Missouri=False, feat_Massachusetts=False, feat_Minnesota=False, feat_Wisconsin=False, feat_Washington=False, feat_NewJersey=False, feat_StateOther=False, feat_PartyOther=False),
 Row(mid='A000367', name='justin-amash', state='Michigan', url='https://www.congress.gov/member/justin-amash/A000367', chamber='House', current_party='Independent', committee

In [155]:
df_nodes_commitee = (
    df_nodes_members.select(F.explode('committee_assignments').alias('id'))
        .union(df_legislation.select(F.explode('committees').alias('id')))
        .dropDuplicates(['id'])
        .withColumnRenamed('id', 'name')
        .coalesce(1)
        .withColumn('id', F.monotonically_increasing_id())
)
print(df_nodes_commitee.count())
df_nodes_commitee.head(5)

95


[Row(name='Education and Labor', id=0),
 Row(name='Foreign Affairs', id=1),
 Row(name='Armed Services, Chair', id=2),
 Row(name='Select Committee on the Modernization of Congress', id=3),
 Row(name='Financial Services', id=4)]

In [156]:
df_nodes_bills = (
    df_legislation
        .withColumnRenamed('bill_id', 'id')
        .dropDuplicates(['id'])
        .withColumnRenamed('id', 'bid')
        .withColumnRenamed('title', 'name')
        .withColumn('timestamp_from', F.unix_timestamp(F.col('date_introduced')))
        .coalesce(1)
        .withColumn('id', F.monotonically_increasing_id())
)
print(df_nodes_bills.count())
df_nodes_bills.show(5)

5806
+------------+--------------------+-------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+------+----------+--------------+---+
|         bid|                name|sponsor|          cosponsors|     related_bills|         policy_area|            subjects|          committees|      bill_progress|             summary|    date_introduced|number| bill_type|timestamp_from| id|
+------------+--------------------+-------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+------+----------+--------------+---+
|H.Con.Res.10|Authorizing the u...|M001205|[M001195, M001187...|[H.R.2500, S.1790]|            Congress|[Conflicts and wa...|[House Administra...|         Introduced|This concurrent r...|2019-01-30 00:00:00|    10|H.Con.Res.|    1548802800|  0|
|H.Con.Res.17|E

In [157]:
area_list = df_nodes_bills.groupby('policy_area').count().filter(F.col('count') > 6).select('policy_area').rdd.flatMap(
    lambda x: x).collect()
print(len(area_list))
print(area_list)

32
['Congress', 'Government Operations and Politics', 'Armed Forces and National Security', 'Arts, Culture, Religion', 'International Affairs', 'Immigration', 'Science, Technology, Communications', 'Public Lands and Natural Resources', 'Crime and Law Enforcement', 'Taxation', 'Commerce', 'Sports and Recreation', 'Labor and Employment', 'Education', 'Environmental Protection', 'Transportation and Public Works', 'Emergency Management', 'Health', 'Foreign Trade and International Finance', 'Economics and Public Finance', 'Civil Rights and Liberties, Minority Issues', 'Families', 'Law', 'Animals', 'Energy', 'Water Resources Development', 'Finance and Financial Sector', 'Native Americans', 'Agriculture and Food', 'Housing and Community Development', 'Social Welfare', None]


In [158]:
df_nodes_bills = (
    df_nodes_bills.select(
        ['*'] + [
            (F.col('policy_area') == area).alias(f'feat_{area}'.replace(' ', '').replace(',', ''))
            for area in area_list
        ]
    )
)
df_nodes_bills.show(5)

+------------+--------------------+-------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+------+----------+--------------+---+-------------+------------------------------------+-----------------------------------+------------------------+-------------------------+----------------+------------------------------------+-----------------------------------+---------------------------+-------------+-------------+------------------------+-----------------------+--------------+----------------------------+---------------------------------+------------------------+-----------+----------------------------------------+------------------------------+------------------------------------------+-------------+--------+------------+-----------+------------------------------+------------------------------+--------------------+-----------------------+-----------------------------------+--

In [159]:
df_nodes_rolecalls = (
    df_rollcall_info
        .withColumnRenamed('rollcall_id', 'id')
        .drop('congress', 'session')
        .dropDuplicates(['id'])
        .withColumnRenamed('question', 'name')
        .withColumnRenamed('id', 'rid')
        .coalesce(1)
        .withColumn('id', F.monotonically_increasing_id())
        .withColumn('feat_ResultP', F.col('result') == 'P')
        .withColumn('feat_ResultF', F.col('result') == 'F')
        .withColumn('feat_ResultA', F.col('result') == 'A')
)
print(df_nodes_rolecalls.count())
df_nodes_rolecalls.head(5)

642


[Row(rid='2019:001', roll_num=1, date=datetime.datetime(2019, 1, 3, 0, 0), bill_id='QUORUM', name='Call by States', result='P', description=None, id=0, feat_ResultP=True, feat_ResultF=False, feat_ResultA=False),
 Row(rid='2019:002', roll_num=2, date=datetime.datetime(2019, 1, 3, 0, 0), bill_id=None, name='Election of the Speaker', result='P', description='\xa0', id=1, feat_ResultP=True, feat_ResultF=False, feat_ResultA=False),
 Row(rid='2019:003', roll_num=3, date=datetime.datetime(2019, 1, 3, 0, 0), bill_id='H.RES.5', name='On Motion to Table the Motion to Refer', result='P', description='Providing for consideration of H.Res. 6, adopting the Rules of the House of Representatives for the 116th Congress; providing for consideration of H.R. 21, making appropriations for the FY ending September 30, 2019, and for other purposes', id=2, feat_ResultP=True, feat_ResultF=False, feat_ResultA=False),
 Row(rid='2019:004', roll_num=4, date=datetime.datetime(2019, 1, 3, 0, 0), bill_id='H.RES.5', na

In [160]:
df_node_ids = (
    df_nodes_members.select('mid')
        .union(df_nodes_commitee.select('id'))
        .union(df_nodes_bills.select('id'))
        .union(df_nodes_rolecalls.select('id'))
        .distinct()
)
print(df_node_ids.count())


def filter_node_ids(df):
    return df.join(
        df_node_ids,
        F.col('src') == F.col('id'),
        'inner'
    ).drop(
        'id'
    ).join(
        df_node_ids,
        F.col('dst') == F.col('id'),
        'inner'
    ).drop('id')

6249


In [161]:
def renumber_edge(df, a, a_id, b, b_id):
    return (
        df.alias('main')
            .join(a.alias('a'), F.col('src') == F.col(f'a.{a_id}'), 'inner')
            .join(b.alias('b'), F.col('dst') == F.col(f'b.{b_id}'), 'inner')
            .select('main.*', 'a.id', 'b.id')
            .withColumn('src', F.col('a.id'))
            .withColumn('dst', F.col('b.id'))
            .drop('id')
    )

In [162]:
df_edges_sponsor = renumber_edge(
    df_legislation.select(
        F.col('sponsor').alias('src'),
        F.col('bill_id').alias('dst'),
        F.unix_timestamp(F.col('date_introduced')).alias('timestamp_from')
    )
        .dropDuplicates(['src', 'dst']),
    df_nodes_members, 'mid',
    df_nodes_bills, 'bid'
)
print(df_edges_sponsor.count())
df_edges_sponsor.show(5)

5806
+---+----+--------------+
|src| dst|timestamp_from|
+---+----+--------------+
|219| 410|    1546470000|
|376|2648|    1546988400|
| 49| 833|    1547679600|
|248|1744|    1548284400|
| 18|1960|    1549407600|
+---+----+--------------+
only showing top 5 rows



In [163]:
df_edges_cosponsor = renumber_edge(
    df_legislation.select(
        F.explode('cosponsors').alias('cosponsor').alias('src'),
        F.col('bill_id').alias('dst'),
        F.unix_timestamp(F.col('date_introduced')).alias('timestamp_from')
    )
        .dropDuplicates(['src', 'dst']),
    df_nodes_members, 'mid',
    df_nodes_bills, 'bid'
)
print(df_edges_cosponsor.count())
df_edges_cosponsor.show(5)

102964
+---+----+--------------+
|src| dst|timestamp_from|
+---+----+--------------+
|247| 388|    1546470000|
|293|1020|    1552345200|
| 24|1540|    1548802800|
| 56|2123|    1546902000|
|166|2123|    1546902000|
+---+----+--------------+
only showing top 5 rows



In [164]:
df_edges_commitee_bill = renumber_edge(
    df_legislation.select(
        F.explode('committees').alias('src'),
        F.col('bill_id').alias('dst'),
        F.unix_timestamp(F.col('date_introduced')).alias('timestamp_from')
    )
        .dropDuplicates(['src', 'dst']),
    df_nodes_commitee, 'name',
    df_nodes_bills, 'bid'
)
print(df_edges_commitee_bill.count())
df_edges_commitee_bill.show(5)

8145
+---+----+--------------+
|src| dst|timestamp_from|
+---+----+--------------+
| 79|1548|    1546470000|
| 83| 328|    1547420400|
| 89|2467|    1547506800|
| 69|1749|    1550790000|
| 71|2289|    1551049200|
+---+----+--------------+
only showing top 5 rows



In [165]:
df_edges_commitee_member = renumber_edge(
    df_nodes_members.select(
        F.col('mid').alias('src'),
        F.explode('committee_assignments').alias('dst'),
    )
        .dropDuplicates(['src', 'dst']),
    df_nodes_members, 'mid',
    df_nodes_commitee, 'name',
)
print(df_edges_commitee_member.count())
df_edges_commitee_member.show(5)

851
+---+---+
|src|dst|
+---+---+
|190| 31|
|342| 31|
|208| 30|
|208|  0|
|208|  4|
+---+---+
only showing top 5 rows



In [166]:
df_edges_related_bill = renumber_edge(
    df_legislation.select(
        F.explode('related_bills').alias('src'),
        F.col('bill_id').alias('dst'),
        F.unix_timestamp(F.col('date_introduced')).alias('timestamp_from')
    )
        .dropDuplicates(['src', 'dst']),
    df_nodes_bills, 'bid',
    df_nodes_bills, 'bid'
)
print(df_edges_related_bill.count())
df_edges_related_bill.show(5)

1885
+----+----+--------------+
| src| dst|timestamp_from|
+----+----+--------------+
|1916| 388|    1546470000|
|2608| 388|    1546470000|
|4376|1741|    1547074800|
|5765|1822|    1548716400|
| 885|1028|    1549580400|
+----+----+--------------+
only showing top 5 rows



In [167]:
df_edges_rollcall_bill = renumber_edge(
    df_rollcall_info.select(
        F.col('rollcall_id').alias('src'),
        F.col('bill_id').alias('dst'),
        F.unix_timestamp(F.col('date')).alias('timestamp_from')
    )
        .dropDuplicates(['src', 'dst']),
    df_nodes_rolecalls, 'rid',
    df_nodes_bills, 'bid'
)
print(df_edges_rollcall_bill.count())
df_edges_rollcall_bill.show(5)

471
+---+----+--------------+
|src| dst|timestamp_from|
+---+----+--------------+
|185| 475|    1546470000|
|547|1585|    1560290400|
|133|1891|    1563141600|
|622|1583|    1571695200|
|459|1585|    1560376800|
+---+----+--------------+
only showing top 5 rows



In [168]:
df_rollcall_votes.head(5)

[Row(name_id='A000374', rollcall_id='2019:641', vote=-1),
 Row(name_id='A000374', rollcall_id='2019:639', vote=1),
 Row(name_id='A000374', rollcall_id='2019:640', vote=-1),
 Row(name_id='A000374', rollcall_id='2019:635', vote=1),
 Row(name_id='A000374', rollcall_id='2019:636', vote=1)]

In [169]:
df_edges_rollcall_vote = renumber_edge(
    df_rollcall_votes.select(
        F.col('name_id').alias('src'),
        F.col('rollcall_id').alias('dst'),
        'vote'
    )
        .dropDuplicates(['src', 'dst']),
    df_nodes_members, 'mid',
    df_nodes_rolecalls, 'rid',
)
print(df_edges_rollcall_vote.count())
df_edges_rollcall_vote.show(5)

284406


+---+---+----+
|src|dst|vote|
+---+---+----+
|189|101|   1|
|189|248|   0|
|189|171|  -1|
|208|519|   1|
|208|541|   1|
+---+---+----+
only showing top 5 rows



In [170]:
df_nodes_members.write.parquet(DS.processed_str('node__Member'), mode='overwrite')
df_nodes_commitee.write.parquet(DS.processed_str('node__Commitee'), mode='overwrite')
df_nodes_bills.write.parquet(DS.processed_str('node__Bill'), mode='overwrite')
df_nodes_rolecalls.write.parquet(DS.processed_str('node__Rollcall'), mode='overwrite')

df_edges_sponsor.write.parquet(DS.processed_str('edge__Member_SPONSORS_Bill'), mode='overwrite')
df_edges_cosponsor.write.parquet(DS.processed_str('edge__Member_COSPONSORS_Bill'), mode='overwrite')
df_edges_commitee_bill.write.parquet(DS.processed_str('edge__Commitee_COMMITEES_Bill'), mode='overwrite')
df_edges_commitee_member.write.parquet(DS.processed_str('edge__Member_ISIN_Commitee'), mode='overwrite')
df_edges_related_bill.write.parquet(DS.processed_str('edge__Bill_RELATEDTO_Bill'), mode='overwrite')
df_edges_rollcall_bill.write.parquet(DS.processed_str('edge__Rollcall_ROLLCALLFOR_Bill'), mode='overwrite')
df_edges_rollcall_vote.write.parquet(DS.processed_str('edge__Member_VOTEDFOR_Rollcall'), mode='overwrite')